In [ ]:
import numpy as np
import pandas as pd
import os
import sqlite3
import pynndescent
import umap
import gzip
from datetime import date
from sklearn.preprocessing import *
from sklearn.pipeline import Pipeline
import sklearn.cluster as cluster
from math import sqrt
from tqdm.notebook import trange, tqdm

from bokeh.plotting import *
from bokeh.transform import factor_cmap
import bokeh.palettes as palettes
from bokeh.models import *
from bokeh.models.tools import *

import networkx as nx
import community
from fa2l import force_atlas2_layout
from fa2 import ForceAtlas2

import sys; 
sys.path.insert(0, '/covid-vis/scripts') #insert scripts file to sys.path

from utils import id_to_path, media_tpl_to_path

In [ ]:
#tweetIDs.txt contains all tweetIds, tweetsdb must be reconstructed.
db = sqlite3.connect('/covid-vis/tweets.db')

In [ ]:
df_users = pd.read_sql_query('select * from users', db)

In [ ]:
df_relationships = pd.read_sql_query('select * from relationships', db)

In [ ]:
# construct the base graph of all users in the db
G = nx.from_pandas_edgelist(df_relationships, source='user_name', target='og_user_name', edge_attr=['tweet_id', 'og_type', 'og_tweet_id'])
print(nx.info(G))

In [ ]:
# copy user metadata from df onto graph nodes
user_attributes = df_users.copy().drop('url', axis=1).set_index('user_name').to_dict('index')

for k, d in G.degree():
    if k in user_attributes:
        user_attributes[k]['degree'] = d
        user_attributes[k]['banned'] = False
    else:
        user_attributes[k] = {'degree': d, 'banned': True}

In [ ]:
for k, c in community.best_partition(G).items():
    user_attributes[k]['community'] = c

In [ ]:
nx.set_node_attributes(G, user_attributes)

In [ ]:
list(enumerate(nx.degree_histogram(G)))

In [ ]:
graphs = []
for i in range(0, 10):
    G_ = G.subgraph([n for n, d in G.degree() if d >= i])
    communities = community.best_partition(G_)
    for k, c in communities.items():
        user_attributes[k]['community'] = c

    nx.set_node_attributes(G_, user_attributes)
    nx.write_graphml(G_, '/covid-vis/network_analysis/G' + str(i) + '.graphml')
    graphs.append(G_)
    print(i)
    print(nx.info(G_))
    print('Number of communities: ' + len(set(communities.values())))
    print()

In [ ]:
# Graph with nodes degree 5 or greater appear to achieve a good balance between major and minor actors/networks
G5 = graphs[5]

In [ ]:
attr5_df = pd.DataFrame.from_dict(G5.nodes(), orient='index').dropna(subset=['id', 'name'], how='all')

In [ ]:
c5summary = attr_df.groupby('community').size().sort_values(ascending=False)

In [ ]:
c5summary.cumsum().to_frame(name="cumsum").reset_index().head(50)

In [ ]:
c5_10 = c5summary.head(10).index
G5_c10 = G5.subgraph([n for n, c in G5.nodes(data='community') if c in c5_10])
print(nx.info(G5_c10))

In [ ]:
nx.write_graphml(G5_c10, '/covid-vis/network_analysis/G5_c10.graphml')

In [ ]:
#Verify community numbers (g_d5_c10) with visible nodes
blue = ['JoeBiden', 'SenWarren', 'joshtpm', 'stengel', 'GeoRebekah', 'GeorgeTakei']
red = ['realDonaldTrump', 'TuckerCarlson', 'CNN', 'NPR']
orange = ['FT', 'jburnmurdoch', 'NicolaSturgeon']
teal = ['AlexBerenson', 'EthicalSkeptic', 'COVID19Tracking', 'GovMikeDeWine']
green = ['ASlavitt', 'nytimes']
purple = ['WHO', 'YouTube']
communities = [blue, red, orange, teal, green, purple]
for comm in communities:
    for user in comm:
        print (user +  ' is in community '  + str(G5_c10.nodes()[str(user)]['community']))

In [ ]:
#find most followed user in each community
followersbycom = {key: 0 for key in c5_10}
popuserbycom = {key: None for key in c5_10}
for name in list(G5_c10.nodes):
    try:
        community_num = G5_c10.nodes()[str(name)]['community']
        followers = G5_c10.nodes()[str(name)]['followers_count']
        if community_num in c5_10:
            if followersbycom[community_num] < followers:
                followersbycom[community_num] = followers
                popuserbycom[community_num] = str(name)
    except:
        pass
print (popuserbycom)

In [ ]:
#returns two item list with average followers in position 0 and average favorites in position 1
totals = {key: [0,0] for key in c5_10}
count = {key: [0,0] for key in c5_10}
averages = {key: [0,0] for key in c5_10}

for name in list(G5_c10.nodes):
    try:
        community_num = G5_c10.nodes()[str(name)]['community']
        followers = G5_c10.nodes()[str(name)]['followers_count']
        favs = G5_c10.nodes()[str(name)]['favourites_count']
        if community_num in c5_10:
            totals[community_num][0] += followers
            count[community_num][0] += 1
            totals[community_num][1] += favs
            count[community_num][1] += 1
    except:
        pass
    
for com in c5_10:
    averages[com][0] = int((totals[com][0])/(count[com][0]))
    averages[com][1] = int((totals[com][1])/(count[com][1]))

print (averages)

In [ ]:
#returns percentage of retweets that are from users in same community
in_network_rts = {key: 0 for key in c5_10}
total = {key: 0 for key in c5_10}

for name in list(G5_c10.nodes):
    try:
        rts = list(G5_c10.adj[str(name)])
        og_comm = int(G5_c10.nodes()[str(name)]['community'])
        for name2 in rts:
            child_comm = int(G5_c10.nodes()[str(name2)]['community'])
            total[og_comm] += 1
            if child_comm == og_comm:
                in_network_rts[og_comm] += 1
    except:
        pass


for key, value in in_network_rts.items():
    print ('Community ' + str(key) +  ' In-Network-Rts as percentage of total: '  + str((in_network_rts[key]/total[key]) * 100) + "%")

In [ ]:
#prints number of verified users in each community
verfd = {key: 0 for key in c5_10}

for name in list(G5_c10.nodes):
    try:
        verified = G5_c10.nodes()[str(name)]['verified']
        community_num = int(G5_c10.nodes()[str(name)]['community'])
        if verified:
            verfd[community_num] += 1
    except:
        pass

print (verfd)

In [ ]:
# for popular communities, returns number of og tweets in that comm, and number of RTs (1.0), replies (3.0), and quotes (2.0)
for key in c5_10:
    print ('Comm ' + key)
    ogs = pd.read_sql_query('select * from tweets where og_user_name in (select user_name from users where g_d5_c10 is ' + key + ')', db)
    print (len(ogs['og_tweet_id'].unique()))
    comm = pd.read_sql_query('select * from tweets where user_name in (select user_name from users where g_d5_c10 is ' + key + ')', db)
    print (comm['og_type'].value_counts())